In [ ]:
base_dir = 'BoPhapDienDienTu'

In [ ]:
from bs4 import BeautifulSoup
from langchain.schema import Document 
import logging 
from tqdm import tqdm
import os
import re
vbpl_dir = os.path.join(base_dir, "vbpl")
def extract_text_from_html(file_path):
    """Extract text from an HTML file using BeautifulSoup"""
    with open(file_path, "r", encoding="utf-8") as f:
        soup = BeautifulSoup(f, "html.parser")

    # Extract text while preserving some structure
    text = soup.get_text(separator="\n").strip()
    text = re.sub(r"\n+", "\n", text)  # Replace multiple newlines with a single one
    text = re.sub(r"\s+", " ", text).strip()
    return text

# Load documents manually
documents = []
for file_name in tqdm(files, desc="Loading documents"):
    file_path = os.path.join(vbpl_dir, file_name)
    try:
        text = extract_text_from_html(file_path)
        if text:
            documents.append(Document(page_content=text, metadata={'source': file_path}))
            print(f"Loaded {file_path}, first 100 chars: {text[:100]}")
        else:
            print(f"Warning: Empty content in {file_path}")
    except Exception as e:
        logging.error(f"Error loading {file_path}: {e}")

print(f"Total loaded documents: {len(documents)}")


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=20, length_function=len)
splits = text_splitter.split_documents(documents)

In [ ]:
# Initialize the HuggingFace embeddings model
from langchain.vectorstores import Chroma
embeddings = HuggingFaceEmbeddings(
    model_name="bkai-foundation-models/vietnamese-bi-encoder",
    model_kwargs={'device': 'cuda' if torch.cuda.is_available() else 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)
# Process the documents and save the vector store
persist_dir = "vectordb"
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings, persist_directory=persist_dir)
vectorstore.persist()

print(f"Processed {len(documents)} documents into {len(splits)} chunks")

In [ ]:
def search_documents(query, k=5):
    """
    Searches for documents similar to the given query and prints the results.
    
    Args:
        query (str): The search query.
        k (int, optional): The number of results to return. Defaults to 5.
    """
    results = vectorstore.similarity_search_with_relevance_scores(query, k=k)
    print(f"\nResults for query: '{query}'\n")
    for i, (doc, score) in enumerate(results, 1):
        print(f"Result {i} (Score: {score:.3f})")
        print(f"Source: {doc.metadata['source']}")
        print(f"Content preview: {doc.page_content[:2000]}...\n")

queries = ["Tìm kiếm các văn bản pháp lý về hợp đồng lao động", "Quy định về bảo vệ môi trường", "Chính sách thuế thu nhập cá nhân"]
for query in queries:
    search_documents(query)
    
"""
Here are the results for the search queries:

Results for query: 'Tìm kiếm các văn bản pháp lý về hợp đồng lao động'

Result 1 (Score: 0.414)
Source: BoPhapDienDienTu\vbpl\full_158152.html
Content preview: động hợp đồng tại các doanh nghiệp, đơn vị sự nghiệp công lập tự chủ về tài chính Quyền lợi của người lao động tại các doanh nghiệp, đơn vị sự nghiệp công lập tự chủ về tài chính thực hiện theo quy định của pháp luật về lao động. Điều 21. Nghĩa vụ của người lao động 1. Thực hiện hợp đồng lao động đã giao kết và các cam kết bảo vệ bí mật Nhà nước (nếu có). 2. Chấp hành kỷ luật lao động, nội quy, quy chế của đơn vị, tuân theo sự điều hành hợp pháp của đơn vị sử dụng lao động. 3. Thực hiện các quy định của pháp luật về bảo hiểm xã hội và pháp luật về bảo hiểm y tế. Điều 22. Giải quyết tranh chấp lao động Quá trình thực hiện hợp đồng lao động phát sinh tranh chấp về quyền và lợi ích của người lao động thì giải quyết theo quy định tại các Điều từ Điều 179 đến Điều 211 Chương XIV Bộ luật Lao động. Thủ trưởng đơn vị có thẩm quyền ký kết hợp đồng lao động là đại diện đơn vị sử dụng lao động hợp đồng tham gia giải quyết tranh chấp khi có tranh chấp lao động xảy ra. Điều 23. Lập, cập nhật, quản lý và sử dụng sổ quản lý lao động 1. Việc lập, cập nhật, quản lý và sử dụng sổ quản lý lao động thực hiện theo Nghị định số 145/2020/NĐ-CP ngày 14 tháng 12 năm 2020 của Chính phủ quy định chi tiết và hướng dẫn thi hành một số điều của Bộ luật Lao động về điều kiện lao động và quan hệ lao động. 2. Sổ quản lý lao động được bảo quản theo chế độ quản lý hồ sơ cán bộ. Điều 24. Kỷ luật lao động, trách nhiệm vật chất 1. Nội quy lao động Người sử dụng lao động phải ban hành nội quy lao động, nếu sử dụng từ 10 người lao động trở lên thì nội quy lao động phải bằng văn bản, nếu sử dụng dưới 10 người lao động thì không bắt buộc ban hành nội quy lao động bằng văn bản nhưng phải thỏa thuận nội dung về kỷ luật lao động, trách nhiệm vật chất trong hợp đồng lao động. Nội dung nội quy lao động thực hiện theo quy định tại khoản 2 Điều 69 Nghị định số 145/2020/NĐ-CP ngày 14 tháng 12 năm 2020 của Chính phủ quy định chi tiết và hướng dẫn thi hành một số điều của Bộ luật Lao động về điều kiện lao động và...

Result 2 (Score: 0.336)
Source: BoPhapDienDienTu\vbpl\full_153913.html
Content preview: động, bảo hiểm xã hội, người lao động Việt Nam đi làm việc ở nước ngoài theo hợp đồng. Chương I QUY ĐỊNH CHUNG Điều 1. Phạm vi điều chỉnh Nghị định này quy định về hành vi vi phạm, hình thức xử phạt, mức xử phạt, biện pháp khắc phục hậu quả, đối tượng xử phạt, thẩm quyền xử phạt, thẩm quyền lập biên bản, thủ tục xử phạt, việc thi hành các hình thức xử phạt, các biện pháp khắc phục hậu quả trong lĩnh vực lao động, bảo hiểm xã hội và người lao động Việt Nam đi làm việc ở nước ngoài theo hợp đồng. Điều 2. Đối tượng áp dụng 1. Người sử dụng lao động, người lao động và các cá nhân, tổ chức khác có hành vi vi phạm hành chính trong lĩnh vực lao động, bảo hiểm xã hội, người lao động Việt Nam đi làm việc ở nước ngoài theo hợp đồng được quy định tại Nghị định này. 2. Người có thẩm quyền xử phạt, thẩm quyền lập biên bản được quy định tại Chương V Nghị định này. 3. Cơ quan, tổ chức, cá nhân khác có liên quan đến việc xử phạt và thi hành các hình thức xử phạt, các biện pháp khắc phục hậu quả quy định trong Nghị định này. Điều 3. Hình thức xử phạt 1. Tổ chức, cá nhân có hành vi vi phạm hành chính trong lĩnh vực lao động, bảo hiểm xã hội, người lao động Việt Nam đi làm việc ở nước ngoài theo hợp đồng thì bị xử phạt theo hình thức xử phạt chính là cảnh cáo hoặc phạt tiền. 2. Căn cứ tính chất, mức độ của hành vi vi phạm, tổ chức, cá nhân có hành vi vi phạm hành chính còn có thể bị áp dụng một hoặc nhiều hình thức xử phạt bổ sung sau đây: a) Tước quyền sử dụng Giấy phép hoạt động cho thuê lại lao động có thời hạn từ 06 tháng đến 12 tháng; b) Tịch thu tang vật, phương tiện vi phạm hành chính là: Giấy phép, Giấy chứng nhận, Chứng chỉ giả mạo; c) Đình chỉ hoạt động huấn luyện an toàn, vệ sinh lao động từ 01 tháng đến 03 tháng; d) Đình chỉ hoạt động kiểm định kỹ thuật an toàn lao động từ 01 tháng đến 03 tháng; đ) Đình chỉ hoạt động quan trắc môi trường lao động từ 03 tháng đến 06 tháng; e) Đình chỉ hoạt động đưa người lao động Việt Nam đi làm việc ở nước ngoài từ 06 tháng đến 12 tháng;...

Result 3 (Score: 0.329)
Source: BoPhapDienDienTu\vbpl\full_152668.html
Content preview: (đối với người lao động sống cùng người sử dụng lao động thuộc đối tượng phải đăng ký tạm trú): ............................................... - Về tạo điều kiện cho người lao động học nghề, học văn hóa:...................................... Điều 8. Kỷ luật lao động - Các trường hợp người sử dụng lao động được áp dụng hình thức khiển trách: ......................................................................................................................... - Các trường hợp người sử dụng lao động được áp dụng hình thức sa thải: ......................................................................................................................... Điều 9. Bồi thường thiệt hại (nếu có) - Các trường hợp người lao động phải bồi thường thiệt hại cho người sử dụng lao động: ............................................... - Các trường hợp người sử dụng lao động phải bồi thường thiệt hại cho người lao động: ..................................................................... Điều 10. Thỏa thuận khác (nếu có) Điều 11. Điều khoản thi hành Hợp đồng này được lập thành 02 bản có giá trị như nhau, mỗi bên giữ 01 bản và có hiệu lực từ ngày tháng năm........................................................ NGƯỜI LAO ĐỘNG (BÊN B) NGƯỜI SỬ DỤNG LAO ĐỘNG (BỀN A) NGƯỜI ĐẠI DIỆN THEO PHÁP LUẬT CỦA NGƯỜI LAO ĐỘNG (đối với người lao động dưới 18 tuổi) - Họ tên:.................................................................................................................. - Địa chỉ nơi cư trú:....................................................................................................... - Điện thoại (nếu có):..................................................................................................... - Số thẻ Căn cước công dân/CMND/Hộ chiếu:....................... cấp ngày......... tại............ - Ký tên: NGƯỜI LÀM CHỨNG (nếu có): - Họ tên:.................................................................................................................. - Số thẻ...

Result 4 (Score: 0.315)
Source: BoPhapDienDienTu\vbpl\full_96115.html
Content preview: nhân dân cấp huyện giải quyết mà tập thể lao động hoặc người sử dụng lao động không đồng ý với quyết định đó hoặc quá thời hạn mà Chủ tịch Ủy ban nhân dân cấp huyện không giải quyết. 3. Tranh chấp liên quan đến lao động bao gồm: a) Tranh chấp về học nghề, tập nghề; b) Tranh chấp về cho thuê lại lao động; c) Tranh chấp về quyền công đoàn, kinh phí công đoàn; d) Tranh chấp về an toàn lao động, vệ sinh lao động. 4. Tranh chấp về bồi thường thiệt hại do đình công bất hợp pháp. 5. Các tranh chấp khác về lao động, trừ trường hợp thuộc thẩm quyền giải quyết của cơ quan, tổ chức khác theo quy định của pháp luật. Điều 33. Những yêu cầu về lao động thuộc thẩm quyền giải quyết của Tòa án 1. Yêu cầu tuyên bố hợp đồng lao động, thỏa ước lao động tập thể vô hiệu. 2. Yêu cầu xét tính hợp pháp của cuộc đình công. 3. Yêu cầu công nhận và cho thi hành tại Việt Nam hoặc không công nhận bản án, quyết định lao động của Tòa án nước ngoài hoặc không công nhận bản án, quyết định lao động của Tòa án nước ngoài không có yêu cầu thi hành tại Việt Nam. 4. Yêu cầu công nhận và cho thi hành tại Việt Nam phán quyết lao động của Trọng tài nước ngoài. 5. Các yêu cầu khác về lao động, trừ trường hợp thuộc thẩm quyền giải quyết của cơ quan, tổ chức khác theo quy định của pháp luật. Điều 34. Thẩm quyền của Tòa án đối với quyết định cá biệt của cơ quan, tổ chức 1. Khi giải quyết vụ việc dân sự, Tòa án có quyền hủy quyết định cá biệt trái pháp luật của cơ quan, tổ chức, người có thẩm quyền xâm phạm quyền, lợi ích hợp pháp của đương sự trong vụ việc dân sự mà Tòa án có nhiệm vụ giải quyết. 2. Quyết định cá biệt quy định tại khoản 1 Điều này là quyết định đã được ban hành về một vấn đề cụ thể và được áp dụng một lần đối với một hoặc một số đối tượng cụ thể. Trường hợp vụ việc dân sự có liên quan đến quyết định này thì phải được Tòa án xem xét trong cùng một vụ việc dân sự đó. 3. Khi xem xét hủy quyết định quy định tại khoản 1 Điều này, Tòa án phải đưa cơ quan, tổ chức hoặc người có thẩm quyền đã ban hành...

Result 5 (Score: 0.306)
Source: BoPhapDienDienTu\vbpl\full_48298.html
Content preview: Đối tượng áp dụng Người lao động; người sử dụng lao động; cơ quan, tổ chức, cá nhân khác có liên quan trực tiếp đến quan hệ lao động theo quy định tại Điều 2 của Bộ luật Lao động. Chương II HỢP ĐỒNG LAO ĐỘNG Mục 1 GIAO KẾT HỢP ĐỒNG LAO ĐỘNG Điều 3. Người giao kết hợp đồng lao động 1. Người giao kết hợp đồng lao động bên phía người sử dụng lao động là người thuộc một trong các trường hợp sau: a) Người đại diện theo pháp luật quy định tại điều lệ của doanh nghiệp, hợp tác xã; b) Người đứng đầu cơ quan, đơn vị, tổ chức theo quy định của pháp luật; c) Chủ hộ gia đình; d) Cá nhân trực tiếp sử dụng lao động. Trường hợp người giao kết hợp đồng lao động bên phía người sử dụng lao động quy định tại các Điểm a, b và c Khoản 1 Điều này không trực tiếp giao kết hợp đồng lao động thì ủy quyền hợp pháp bằng văn bản cho người khác giao kết hợp đồng lao động theo mẫu do Bộ Lao động - Thương binh và Xã hội quy định. 2. Người giao kết hợp đồng lao động bên phía người lao động là người thuộc một trong các trường hợp sau: a) Người lao động từ đủ 18 tuổi trở lên; b) Người lao động chưa thành niên từ đủ 15 tuổi đến dưới 18 tuổi và có sự đồng ý bằng văn bản của người đại diện theo pháp luật của người lao động; c) Người đại diện theo pháp luật đối với người dưới 15 tuổi và có sự đồng ý của người dưới 15 tuổi; d) Người lao động được những người lao động trong nhóm ủy quyền hợp pháp giao kết hợp đồng lao động. 3. Người được ủy quyền giao kết hợp đồng lao động quy định tại Khoản 1 và Khoản 2 Điều này không được tiếp tục ủy quyền cho người khác giao kết hợp đồng lao động. Điều 4. Nội dung hợp đồng lao động Nội dung chủ yếu của hợp đồng lao động tại Khoản 1 Điều 23 của Bộ luật Lao động được quy định như sau: 1. Tên và địa chỉ của người sử dụng lao động được quy định như sau: a) Tên doanh nghiệp, cơ quan, tổ chức, hợp tác xã, hộ gia đình thuê mướn, sử dụng lao động theo hợp đồng lao động theo giấy chứng nhận đăng ký doanh nghiệp, hợp tác xã hoặc giấy chứng nhận đầu tư hoặc quyết định thành lập...


Results for query: 'Quy định về bảo vệ môi trường'

Result 1 (Score: 0.524)
Source: BoPhapDienDienTu\vbpl\full_146609.html
Content preview: BẢO VỆ MÔI TRƯỜNG Điều 164. Nội dung quản lý nhà nước về bảo vệ môi trường 1. Ban hành và tổ chức thực hiện chính sách, pháp luật; tiêu chuẩn, quy chuẩn kỹ thuật, hướng dẫn kỹ thuật; chiến lược, quy hoạch, kế hoạch; chương trình, đề án, dự án về bảo vệ môi trường. 2. Thẩm định, phê duyệt kết quả thẩm định báo cáo đánh giá tác động môi trường; cấp, cấp đổi, điều chỉnh, cấp lại, thu hồi giấy phép môi trường; đăng ký môi trường; cấp, cấp lại, thu hồi giấy chứng nhận về môi trường. 3. Kiểm soát nguồn ô nhiễm; quản lý chất thải, chất lượng môi trường; cải tạo và phục hồi môi trường; bảo vệ môi trường di sản thiên nhiên, bảo tồn thiên nhiên và đa dạng sinh học; phòng ngừa, ứng phó sự cố môi trường. 4. Xây dựng, quản lý hệ thống quan trắc môi trường; tổ chức quan trắc môi trường. 5. Xây dựng, cập nhật hệ thống thông tin, cơ sở dữ liệu về môi trường, báo cáo về môi trường. 6. Xây dựng và triển khai hệ thống giám sát và đánh giá các hoạt động thích ứng với biến đổi khí hậu; hệ thống đo đạc, báo cáo, thẩm định giảm nhẹ phát thải khí nhà kính. 7. Kiểm kê khí nhà kính; xây dựng và cập nhật kịch bản, cơ sở dữ liệu về biến đổi khí hậu, nước biển dâng và ngập lụt đô thị; đánh giá khí hậu quốc gia; hướng dẫn việc sử dụng thông tin, dữ liệu về biến đổi khí hậu và lồng ghép nội dung ứng phó với biến đổi khí hậu vào chiến lược, quy hoạch. 8. Tổ chức thực hiện thị trường các-bon trong nước; thực hiện cơ chế trao đổi tín chỉ, cam kết quốc tế về giảm nhẹ phát thải khí nhà kính. 9. Thanh tra, kiểm tra; giải quyết khiếu nại, tố cáo; xử lý vi phạm pháp luật về bảo vệ môi trường; xác định thiệt hại và yêu cầu bồi thường thiệt hại về môi trường. 10. Truyền thông, giáo dục môi trường, nâng cao nhận thức, ý thức về bảo vệ môi trường; đào tạo, bồi dưỡng chuyên môn, nghiệp vụ quản lý về bảo vệ môi trường. 11. Tổ chức nghiên cứu khoa học, phát triển công nghệ, ứng dụng chuyển giao công nghệ, hội nhập và hợp tác quốc tế về bảo vệ môi trường. 12. Bố trí kinh phí ngân sách nhà nước để thực hiện...

Result 2 (Score: 0.455)
Source: BoPhapDienDienTu\vbpl\full_146609.html
Content preview: với các Bộ, cơ quan ngang Bộ, tổ chức chính trị - xã hội, cơ quan truyền thông, báo chí có trách nhiệm truyền thông, phổ biến kiến thức, tuyên truyền pháp luật về bảo vệ môi trường. 3. Bộ, cơ quan ngang Bộ chủ trì, phối hợp với Bộ Tài nguyên và Môi trường, các cơ quan truyền thông, báo chí có trách nhiệm truyền thông, phổ biến kiến thức, tuyên truyền pháp luật về bảo vệ môi trường thuộc lĩnh vực quản lý. 4. Ủy ban nhân dân cấp tỉnh chủ trì, phối hợp với các cơ quan truyền thông, báo chí có trách nhiệm truyền thông, phổ biến kiến thức, tuyên truyền pháp luật về bảo vệ môi trường trên địa bàn. Chương XII HỘI NHẬP VÀ HỢP TÁC QUỐC TẾ VỀ BẢO VỆ MÔI TRƯỜNG Điều 155. Nguyên tắc hội nhập và hợp tác quốc tế về bảo vệ môi trường 1. Hội nhập và hợp tác quốc tế về bảo vệ môi trường được thực hiện trên cơ sở bình đẳng, cùng có lợi, tăng cường sức mạnh tổng hợp và nâng cao vị thế, uy tín của quốc gia, tôn trọng độc lập chủ quyền và toàn vẹn lãnh thổ, tuân thủ pháp luật của mỗi bên, pháp luật quốc tế và cam kết trong các điều ước quốc tế, thỏa thuận quốc tế có liên quan đến môi trường. 2. Điều ước quốc tế, thỏa thuận quốc tế có lợi cho việc bảo vệ môi trường của quốc gia, khu vực và toàn cầu, phù hợp với lợi ích và năng lực của Việt Nam được ưu tiên xem xét để ký kết. 3. Tranh chấp quốc tế liên quan đến môi trường được giải quyết thông qua các biện pháp hoà bình, theo thông lệ, pháp luật quốc tế và pháp luật của các bên liên quan. Điều 156. Trách nhiệm trong hội nhập và hợp tác quốc tế về bảo vệ môi trường 1. Nhà nước khuyến khích việc chủ động hội nhập quốc tế về bảo vệ môi trường, tập trung cho các lĩnh vực quản lý và bảo vệ các thành phần môi trường, bảo tồn đa dạng sinh học, tăng trưởng xanh, phát triển bền vững và ứng phó với biến đổi khí hậu; bảo đảm về nguồn lực và thực hiện đầy đủ nghĩa vụ đã cam kết trong điều ước quốc tế, thỏa thuận quốc tế có liên quan đến môi trường, đáp ứng xu thế hội nhập quốc tế, hỗ trợ cho hội nhập quốc tế về kinh tế. 2. Nhà nước khuyến khích đầu...

Result 3 (Score: 0.450)
Source: BoPhapDienDienTu\vbpl\full_146609.html
Content preview: chất thải, tăng cường tái sử dụng, tái chế chất thải để khai thác giá trị tài nguyên của chất thải. 5. Bảo vệ môi trường phải phù hợp với quy luật, đặc điểm tự nhiên, văn hóa, lịch sử, cơ chế thị trường, trình độ phát triển kinh tế - xã hội; thúc đẩy phát triển vùng đồng bào dân tộc thiểu số và miền núi. 6. Cơ quan, tổ chức, cộng đồng dân cư, hộ gia đình và cá nhân được hưởng lợi từ môi trường có nghĩa vụ đóng góp tài chính cho hoạt động bảo vệ môi trường; gây ô nhiễm, sự cố và suy thoái môi trường phải chi trả, bồi thường thiệt hại, khắc phục, xử lý và chịu trách nhiệm khác theo quy định của pháp luật. 7. Hoạt động bảo vệ môi trường bảo đảm không gây phương hại chủ quyền, an ninh và lợi ích quốc gia, gắn liền với bảo vệ môi trường khu vực và toàn cầu. Điều 5. Chính sách của Nhà nước về bảo vệ môi trường 1. Tạo điều kiện thuận lợi cho cơ quan, tổ chức, cộng đồng dân cư, hộ gia đình và cá nhân tham gia thực hiện, kiểm tra, giám sát hoạt động bảo vệ môi trường. 2. Tuyên truyền, giáo dục kết hợp với biện pháp hành chính, kinh tế và biện pháp khác để tăng cường việc tuân thủ pháp luật về bảo vệ môi trường, xây dựng văn hóa bảo vệ môi trường. 3. Chú trọng bảo tồn đa dạng sinh học, bảo vệ môi trường di sản thiên nhiên; khai thác, sử dụng hợp lý và tiết kiệm tài nguyên thiên nhiên; phát triển năng lượng sạch và năng lượng tái tạo; phát triển hạ tầng kỹ thuật bảo vệ môi trường. 4. Ưu tiên xử lý ô nhiễm môi trường, phục hồi hệ sinh thái tự nhiên bị suy thoái, chú trọng bảo vệ môi trường khu dân cư. 5. Đa dạng hóa các nguồn vốn đầu tư cho bảo vệ môi trường; bố trí khoản chi riêng cho bảo vệ môi trường trong ngân sách nhà nước với tỷ lệ tăng dần theo khả năng của ngân sách nhà nước và yêu cầu, nhiệm vụ bảo vệ môi trường; ưu tiên nguồn kinh phí cho các nhiệm vụ trọng điểm về bảo vệ môi trường. 6. Bảo đảm quyền lợi của tổ chức, cộng đồng dân cư, hộ gia đình và cá nhân đóng góp cho hoạt động bảo vệ môi trường; ưu đãi, hỗ trợ hoạt động bảo vệ môi trường; thúc đẩy sản phẩm, dịch...

Result 4 (Score: 0.449)
Source: BoPhapDienDienTu\vbpl\full_132982.html
Content preview: luật về bảo vệ môi trường. 3. Việc lập, thẩm định, phê duyệt, công bố, tổ chức thực hiện quy hoạch thăm dò, khai thác, chế biến và sử dụng các loại khoáng sản, quy hoạch thăm dò, khai thác, chế biến và sử dụng quặng phóng xạ, quy hoạch thăm dò, khai thác, chế biến và sử dụng các loại khoáng sản làm vật liệu xây dựng phải tuân thủ quy định của pháp luật về quy hoạch.”. 6. Sửa đổi, bổ sung Điều 14 như sau: “Điều 14. Điều chỉnh quy hoạch khoáng sản Việc điều chỉnh quy hoạch khoáng sản được thực hiện khi có một trong các căn cứ sau đây: 1. Có một trong các căn cứ điều chỉnh quy hoạch theo quy định của pháp luật về quy hoạch; 2. Có phát hiện mới về khoáng sản làm ảnh hưởng đến tính chất, nội dung quy hoạch; 3. Khi xảy ra trường hợp quy định tại khoản 4 Điều 28 của Luật này.”. 7. Sửa đổi, bổ sung khoản 3 Điều 17 như sau: “3. Cơ quan tổ chức lập quy hoạch xây dựng vùng liên huyện, quy hoạch xây dựng vùng huyện, quy hoạch chung đô thị, quy hoạch xây dựng điểm dân cư nông thôn khi trình phê duyệt quy hoạch phải trình kèm theo ý kiến bằng văn bản của cơ quan quản lý nhà nước có thẩm quyền cấp giấy phép quy định tại Điều 82 của Luật này.”. 8. Sửa đổi, bổ sung khoản 1 Điều 26 như sau: “1. Khu vực hoạt động khoáng sản là khu vực có khoáng sản đã được điều tra cơ bản địa chất về khoáng sản và được cơ quan nhà nước có thẩm quyền khoanh định trong quy hoạch có liên quan theo quy định của pháp luật về quy hoạch.”. 9. Sửa đổi, bổ sung khoản 4 Điều 28 như sau: “4. Trường hợp cần thăm dò, khai thác khoáng sản ở khu vực cấm hoạt động khoáng sản, khu vực tạm thời cấm hoạt động khoáng sản, cơ quan quản lý nhà nước có thẩm quyền cấp giấy phép quy định tại Điều 82 của Luật này phải báo cáo Thủ tướng Chính phủ xem xét, quyết định việc điều chỉnh quy hoạch có liên quan theo quy định của pháp luật về quy hoạch.”. 10. Sửa đổi, bổ sung điểm b khoản 2 Điều 40 như sau: “b) Có đề án thăm dò phù hợp với quy hoạch có liên quan theo quy định của pháp luật về quy hoạch; đối với khoáng sản độc hại còn...

Result 5 (Score: 0.425)
Source: BoPhapDienDienTu\vbpl\full_45966.html
Content preview: Điều 6. Trách nhiệm bảo vệ môi trường của du khách 1. Giữ gìn vệ sinh, bảo vệ cảnh quan môi trường, không có những hành vi làm ảnh hưởng xấu đến cảnh quan tại cơ sở. 2. Không mang hóa chất độc hại, chất nổ, chất dễ gây cháy, các chất hôi thối, ô nhiễm, đến nơi tham quan, du lịch 3. Không đốt lửa, tạo tia lửa điện, nguồn gần nhiệt dễ cháy trong các khu vực dễ gây cháy rừng hoặc huỷ hoại thảm thực vật. 4. Dâng lễ, đặt tiền lễ, tiền giọt dầu, thắp hương, hóa sớ, bỏ rác thải đúng nơi quy định; không xâm hại đến di tích, di vật, không viết, vẽ, ký hiệu riêng làm ảnh hưởng đến mỹ quan của những bộ phận cấu thành của di tích. 5. Không mua bán, sử dụng động vật, thực vật quý hiếm và sản phẩm có nguồn gốc từ động thực vật quý hiếm theo quy định; không mang các loại sinh vật ngoại lai gây nguy hại đối với môi trường, con người vào cơ sở. 6 Kịp thời phản ánh với người có trách nhiệm tại cơ sở những hành vi vi phạm quy định về bảo vệ môi trường. 7. Tham gia các phong trào bảo vệ môi trường, tuyên truyền, vận động mọi người cùng chấp hành tốt các quy định về bảo vệ môi trường tại cơ sở; sử dụng sản phẩm thân thiện với môi trường. 8. Thực hiện các quy định khác có liên quan của pháp luật về bảo vệ môi trường. Điều 7. Nhà vệ sinh tại cơ sở 1. Tổ chức, cá nhân quản lý và hoạt động sản xuất, kinh doanh tại cơ sở phải xây dựng, bố trí hợp lý nhà vệ sinh phục vụ du khách, tối thiểu phải bảo đảm theo tiêu chuẩn nhà tiêu tại QCVN 01:2011/BYT - Quy chuẩn kỹ thuật quốc gia về nhà tiêu - Điều kiện bảo đảm hợp vệ sinh được ban hành kèm theo Thông tư số 27/2011/TT-BYT ngày 24 tháng 6 năm 2011 của Bộ Y tế. 2. Có biển chỉ dẫn nhà vệ sinh rõ ràng, bằng tiếng Việt và tiếng Anh (kèm chữ viết tắt tiếng Anh: WC), ở những nơi mang tính đặc thù cần thiết có thể có thêm ngôn ngữ khác. 3. Có vị trí thuận lợi, dễ tiếp cận, bảo đảm an ninh, an toàn cho khách khi sử dụng, phù hợp với môi trường cảnh quan; có khu vực vệ sinh riêng cho nam và nữ, có ký hiệu bằng hình ảnh để phân biệt nhà vệ sinh...


Results for query: 'Chính sách thuế thu nhập cá nhân'

Result 1 (Score: 0.435)
Source: BoPhapDienDienTu\vbpl\full_51578.html
Content preview: cá nhân và Luật sửa đổi, bổ sung một số điều của Luật Thuế thu nhập cá nhân, Điều 3 Nghị định số 91/2014/NĐ-CP ngày 01 tháng 10 năm 2014 sửa đổi, bổ sung một số điều tại các Nghị định quy định về thuế như sau: 1. Sửa đổi, bổ sung Khoản 1 Điều 3 như sau: “1. Thu nhập từ kinh doanh, bao gồm: a) Thu nhập từ hoạt động sản xuất, kinh doanh hàng hóa, dịch vụ theo quy định của pháp luật. Riêng đối với thu nhập từ hoạt động sản xuất nông nghiệp, lâm nghiệp, làm muối, nuôi trồng, đánh bắt thủy sản chỉ áp dụng đối với trường hợp không đủ điều kiện được miễn thuế quy định tại Khoản 5 Điều 4 Nghị định này. b) Thu nhập từ hoạt động hành nghề độc lập của cá nhân có giấy phép hoặc chứng chỉ hành nghề theo quy định của pháp luật. Thu nhập từ kinh doanh quy định tại Khoản này không bao gồm thu nhập của cá nhân kinh doanh có doanh thu từ 100 triệu đồng/năm trở xuống.” 2. Sửa đổi Điểm đ Khoản 2 Điều 3 như sau: “đ) Các khoản lợi ích bằng tiền hoặc không bằng tiền ngoài tiền lương, tiền công do người sử dụng lao động trả mà người nộp thuế được hưởng dưới mọi hình thức: - Tiền nhà ở, điện, nước và các dịch vụ kèm theo (nếu có), không bao gồm khoản lợi ích về nhà ở, điện, nước và các dịch vụ kèm theo (nếu có) đối với nhà ở do người sử dụng lao động xây dựng để cung cấp cho người lao động làm việc tại khu công nghiệp hoặc nhà ở do người sử dụng lao động xây dựng tại khu kinh tế, địa bàn có điều kiện kinh tế xã hội khó khăn hoặc địa bàn có điều kiện kinh tế xã hội đặc biệt khó khăn cung cấp cho người lao động. - Khoản tiền do người sử dụng lao động mua bảo hiểm nhân thọ, bảo hiểm không bắt buộc khác có tích lũy về phí bảo hiểm, mua bảo hiểm hưu trí tự nguyện hoặc đóng góp Quỹ hưu trí tự nguyện cho người lao động. - Phí hội viên và các khoản chi dịch vụ khác phục vụ cho cá nhân theo yêu cầu, như: Chăm sóc sức khỏe, vui chơi, thể thao, giải trí, thẩm mỹ. - Các khoản lợi ích khác theo quy định của pháp luật.” 3. Sửa đổi, bổ sung Điểm b Khoản 2 Điều 3 như sau: “b) Các khoản phụ cấp, trợ cấp,...

Result 2 (Score: 0.349)
Source: BoPhapDienDienTu\vbpl\full_32504.html
Content preview: ch ế . Điều 32. Tỷ lệ khấu trừ một phần tiền lương hoặc một phần thu nhập đối với cá nhân 1. Chỉ khấu trừ một phần tiền lương, tiền công hoặc một phần thu nhập tương ứng với số tiền đã ghi trong quyết định hành chính thuế của người có thẩm quyền. 2. Tỷ lệ khấu trừ tiền lương, tiền công hoặc thu nhập đối với cá nhân không thấp hơn 10% và không quá 30% t ổ ng số tiền lương, trợ cấp hàng tháng của cá nhân đó; đối với những khoản thu nhập khác thì tỷ lệ khấu trừ căn cứ vào thu nhập thực tế, nhưng không quá 50% tổng số thu nhập. Điều 33. Trách nhiệm của cơ quan, tổ chức, người sử dụng lao động đang quản lý tiền lương, tiền công hoặc thu nhập của cá nhân bị cưỡng chế Cơ quan, tổ chức đang quản lý tiền lương hoặc thu nhập của đối tượng bị cưỡng chế có trách nhiệm: 1. Khấu trừ một phần tiền lương hoặc thu nhập của đối tượng bị cưỡng chế và chuy ể n số tiền đã khấu trừ vào ngân sách nhà nước theo nội dung ghi trong quyết định cưỡng chế thi hành quyết định hành chính thuế, kể từ kỳ trả tiền lương hoặc thu nhập gần nhất cho đến khi khấu trừ đủ số tiền thuế, tiền chậm nộp tiền thuế, tiền phạt, tiền chậm nộp tiền phạt theo quyết định cưỡng chế thi hành quyết định hành chính thuế, đồng thời thông báo cho người ra quyết định cưỡng chế và đối tượng bị cưỡng chế biết; 2. Khi đến kỳ phát tiền lương hoặc thu nhập gần nhất, cơ quan, tổ chức, người đang quản lý tiền lương hoặc thu nhập của cá nhân bị cưỡng chế có trách nhiệm kh ấ u trừ một ph ầ n ti ề n lương hoặc thu nhập của cá nhân bị cưỡng chế theo nội dung ghi trong quyết định cưỡng chế và chuyển số tiền đã khấu trừ vào ngân sách nhà nước đã ghi tr ong quyết định cưỡng chế, đồng thời thông báo cho người có thẩm quyền ra quyết định cưỡng chế biết; 3. Trường h ợ p chưa khấu trừ đủ số tiền thuế, tiền chậm nộp tiền thuế, số tiền phạt, tiền chậm nộp tiền phạt theo quyết định cưỡng chế mà hợp đồng lao động của đối tượng bị cưỡng chế chấm dứt, cơ quan, tổ chức sử dụng lao động phải thông báo cho người ra quyết định c ưỡ ng chế biết trong...

Result 3 (Score: 0.332)
Source: BoPhapDienDienTu\vbpl\full_46289.html
Content preview: ngăn ngừa việc trốn lậu thuế đối với các loại thuế đánh vào thu nhập và là cá nhân cư trú tại Việt Nam thì nghĩa vụ thuế thu nhập cá nhân được tính từ tháng đến Việt Nam trong trường hợp cá nhân lần đầu tiên có mặt tại Việt Nam đến tháng kết thúc hợp đồng lao động và rời Việt Nam (được tính đủ theo tháng) không phải thực hiện các thủ tục xác nhận lãnh sự để được thực hiện không thu thuế trùng hai lần theo Hiệp định tránh đánh thuế trùng giữa hai quốc gia. Đối với cá nhân không cư trú, thu nhập chịu thuế là thu nhập phát sinh tại Việt Nam, không phân biệt nơi trả và nhận thu nhập” Điều 3. Sửa đổi, bổ sung Thông tư số 219/2013/TT-BTC ngày 31/12/2013 của Bộ Tài chính hướng dẫn thi hành Luật Thuế giá trị gia tăng và Nghị định số 209/2013/NĐ-CP ngày 18/12/2013 của Chính phủ quy định chi tiết và hướng dẫn thi hành một số điều Luật Thuế giá trị gia tăng như sau: 1. Bổ sung điểm g khoản 7 Điều 5 Thông tư số 219/2013/TT-BTC như sau: “g) Cơ sở kinh doanh không phải nộp thuế GTGT khâu nhập khẩu trong trường hợp nhập khẩu hàng hóa đã xuất khẩu nhưng bị phía nước ngoài trả lại. Khi cơ sở kinh doanh bán hàng hóa bị trả lại này trong nội địa thì phải kê khai, nộp thuế GTGT theo quy định”. 2. Sửa đổi, bổ sung khoản 4 Điều 7 Thông tư số 219/2013/TT-BTC như sau: “4. Giá tính thuế đối với sản phẩm, hàng hóa, dịch vụ tiêu dùng nội bộ. Hàng hóa luân chuyển nội bộ như hàng hóa được xuất để chuyển kho nội bộ, xuất vật tư, bán thành phẩm, để tiếp tục quá trình sản xuất trong một cơ sở sản xuất, kinh doanh hoặc hàng hóa, dịch vụ do cơ sở kinh doanh xuất hoặc cung ứng sử dụng phục vụ hoạt động kinh doanh thì không phải tính, nộp thuế GTGT. Trường hợp cơ sở kinh doanh tự sản xuất, xây dựng tài sản cố định (tài sản cố định tự làm) để phục vụ sản xuất, kinh doanh hàng hóa, dịch vụ chịu thuế GTGT thì khi hoàn thành, nghiệm thu, bàn giao, cơ sở kinh doanh không phải lập hóa đơn. Thuế GTGT đầu vào hình thành nên tài sản cố định tự làm được kê khai, khấu trừ theo quy định. Trường hợp xuất máy móc,...

Result 4 (Score: 0.317)
Source: BoPhapDienDienTu\vbpl\full_27546.html
Content preview: thu nhập cá nhân đối với cá nhân có thu nhập tính thuế từ tiền lương, tiền công đến mức phải chịu thuế thu nhập cá nhân ở bậc 1 (thu nhập tính thuế nhỏ hơn hoặc bằng 5 triệu đồng/tháng) của Biểu lũy tiến từng phần quy định tại Điều 22 Luật Thuế thu nhập cá nhân. Đối với các cá nhân có thu nhập tính thuế từ bậc 2 trở lên (thu nhập tính thuế lớn hơn 5 triệu đồng/tháng), tổ chức trả thu nhập thực hiện khai thuế, khấu trừ, nộp thuế thu nhập cá nhân từ bậc 1 của Biểu thuế luỹ tiến từng phần quy định tại Điều 22 Luật Thuế thu nhập cá nhân . Tổ chức, cá nhân trả thu nhập ghi số thuế đã khấu trừ theo mức đã được miễn giảm vào chỉ tiêu [33], [34], [35] của Tờ khai mẫu số 02/KK-TNCN ban hành kèm theo Thông tư số 28/2011/TT-BTC ngày 28/02/2011 của Bộ Tài chính. Ví dụ 1 . Anh A có thu nhập từ tiền lương, tiền công của tháng 8/2011 là 9.200.000 đồng. Anh A chỉ có giảm trừ cho bản thân là 4.000.000 đồng, các khoản đóng bảo hiểm là 200.000 đồng, không có đóng góp từ thiện nhân đạo, không có giảm trừ cho người phụ thuộc. Thu nhập tính thuế của Anh A là: 9.200.000 đồng - 4.000.000 đồng - 200.000 đồng = 5.000.000 đồng. Anh A có thu nhập tính thuế ở bậc 1 của Biểu thuế luỹ tiến từng phần nên anh A tạm thời chưa phải nộp thuế Thu nhập cá nhân của tháng 8/2011. Ví dụ 2. Anh B có thu nhập từ tiền lương, tiền công của tháng 8/2011 là 21.200.000 đồng. Anh B chỉ có giảm trừ cho bản thân là 4.000.000 đồng, các khoản đóng bảo hiểm là 318.000 đồng, không có đóng góp từ thiện nhân đạo, không có giảm trừ cho người phụ thuộc. Thu nhập tính thuế của Anh B là: 21.200.000 đồng - 4.000.000 đồng - 318.000 đồng = 16.882.000 đồng. Anh B có thu nhập tính thuế từ bậc 2 trở lên (đến bậc 3) của Biểu thuế luỹ tiến từng phần nên anh B phải nộp thuế Thu nhập cá nhân theo quy định tại Điều 22 Luật Thuế thu nhập cá nhân : Số thuế Thu nhập cá nhân phải nộp của Anh B được tính là: + Bậc 1: 5.000.000 đồng x 5% = 250.000 đồng. + Bậc 2: (10.000.000 đồng – 5.000.000 đ ) x 10% = 500.000 đồng. + Bậc 3: (16.882.000 đồng...

Result 5 (Score: 0.315)
Source: BoPhapDienDienTu\vbpl\full_30676.html
Content preview: quản lý thu theo quy định của pháp luật. Điều 2. Người nộp thuế Người nộp thuế quy định tại Nghị định này bao gồm: 1. Tổ chức, hộ gia đình, cá nhân nộp thuế, phí, lệ phí và các khoản thu khác thuộc ngân sách nhà nước do cơ quan quản lý thuế quản lý thu theo quy định của pháp luật. 2. Tổ chức được giao nhiệm vụ thu phí, lệ phí thuộc ngân sách nhà nước. 3. Tổ chức, cá nhân khấu trừ thuế; tổ chức, cá nhân làm thủ tục về thuế thay người nộp thuế bao gồm: a) Tổ chức, cá nhân là bên Việt Nam ký hợp đồng với tổ chức, cá nhân nước ngoài kinh doanh tại Việt Nam không theo quy định của pháp luật v ề đầu tư và không thực hiện chế độ kế toán Việt Nam; b) Tổ chức, cá nhân khấu trừ thuế khi chi trả thu nhập cho người có thu nhập thuộc diện nộp thuế thu nhập cá nhân; c) Tổ chức làm đại lý tàu biển, đại lý cho các hãng vận tải nước ngoài có trách nhiệm khấu trừ thuế thu nhập doanh nghiệp đối với hoạt động kinh doanh vận chuyển hàng hóa bằng tàu biển từ các cảng biển Việt Nam ra nước ngoài hoặc giữa các cảng biển Việt Nam; d) Tổ chức kinh doanh dịch vụ làm thủ tục về thuế; đ) Đại lý làm thủ tục hải quan đối với hàng hóa xuất khẩu, nhập khẩu; e) Tổ chức, cá nhân cung cấp dịch vụ bưu chính, dịch vụ chuyển phát nhanh quốc tế trong trường hợp nộp thay thuế cho tổ chức, cá nhân nộp thuế; g) Tổ chức tín dụng theo quy định của Luật C ác tổ chức tín dụng trong trường hợp bảo lãnh nộp thuế cho tổ chức, cá nhân nộp thuế. Điều 3. Ủ y nhiệm thu thuế 1. Cơ quan quản lý thuế ủy nhiệm cho cơ quan, tổ chức, cá nhân thu một số loại thuế theo quy định của Bộ Tài chính. 2. Việc ủ y nhiệm thu thuế phải được thực hiện thông qua hợp đồng giữa Thủ trưởng cơ quan quản lý thuế với cơ quan, tổ chức, cá nhân được ủy nhiệm thu, trừ một số trường hợp ủy nhiệm thu đối với các khoản thu nhập có tính chất không thường xuyên theo quy định của Bộ Tài chính. 3. Bên được ủy nhiệm thu thuế có trách nhiệm: Thông báo và đôn đốc người nộp thuế thực hiện nộp thuế theo hợp đồng ủy nhiệm thu; cấp chứng từ thu tiền cho người...


"""    